In [1]:
using Toms566

In [2]:
function newtmin_tom(p, x0; max_its = 100, opt_tol = 1e-6, rho = 0.9, mu = 1e-4, delta = 1e-8, BFGS = false)
    """
    p = Problem taken from Toms566 package
     p.x0 = standard initial guess
     p.obj = oracle returning objective function values
     p.grd = oracle returning gradient of obj
     p.hes = oracle returning hessian of obj
    x0 = starting point (if None, use p.x0)
    max_its = maximum number of iterations to try before quitting ( = 100 if not specified)
    opt_tol is tolerance defining the stopping condition |p.grd(x)| < opt_tol*|p.grd(x0)| ( = 1e-6 if not specified)
    rho = backtracking linesearch scaling constant
    mu = coefficient for Armijo condition
    delta = positive constant to replace negative (or zero) singular values of the Hessian
    TO DO:
        - add BFGS search direction option
    """
    x = x0; n = length(x0)
    f = p.obj(x0)
    g = p.grd(x0)
    g0 = p.grd(x0)
    its = 0
    count_notpd = 0
    if ~(BFGS)
        while (its < max_its) & (norm(g) > opt_tol * norm(g0)) # stopping criteria
            # compute function value, gradient, and hessian at current point
            f = p.obj(x)
            g = p.grd(x)
            H = p.hes(x)
            # try Cholesky factorization to determine if H is positive definite
            try R = chol(H)
                # if H is positive definite, compute the Newton search direction d = -H\g
                d = -H\g
                # use backtracking to compute an acceptable step length
                a = 1
                while p.obj(x+a*d) > f + a*mu*dot(g,d)
                    a = rho*a
                end
                # update x value using the step length a that we just decided on
                x = x + a*d
            catch
                count_notpd += 1
                # H is not positive definite, compute eigenvalue decomposition
                (l,Q) = eig(H)
                # flip and scale negative eigenvalues by delta > 0
                l = max(l,-delta*l)
                # compute search direction by inverting modified Hessian
                B = Q*diagm(l)*Q'
                d = -B\g
                a = 1
                while p.obj(x+a*d) > f + a*mu*dot(g,d)
                    a = rho*a
                end
                x = x + a*d
            end
            its += 1
        end
    else
        # Compute BFGS search direction
        # initialze B matrix to the identity
        B = eye(n)
        while (its < max_its) & (norm(g) > opt_tol * norm(g0))
            f = p.obj(x)
            g = p.grd(x)
            d = -B\g
            a = 1
            while p.obj(x+a*d) > f + mu*a*dot(d,g)
                a = a*rho
            end
            s = a*d
            x = x + s
            y = p.grd(x) - g
            # BFGS update
            B = B + y*y'/dot(y,s) - B*s*s'*B/dot(s,B*s)
            its += 1
        end
    end        
    return (x,its)
end

newtmin_tom (generic function with 1 method)

In [20]:
res_table = cell(18,4)
for n = 1:18
    p = Problem(n)
    (x_newt,its_newt) = newtmin_tom(p,p.x0,max_its = 500)
    fmin_newt = p.obj(x_newt)
    res_table[n,1:2] = [fmin_newt;its_newt]
    (x_bfgs,its_bfgs) = newtmin_tom(p,p.x0,max_its = 500,BFGS=true)
    fmin_bfgs = p.obj(x_bfgs)
    res_table[n,3:4] = [fmin_bfgs;its_bfgs]
end

In [21]:
res_table

18x4 Array{Any,2}:
    8.41724e-29   14.0      2.29068e-13   28.0
  NaN              3.0      8.10366e-13  131.0
    1.12793e-8     3.0      1.12793e-8     6.0
    1.04153e-8    14.0      2.25191e-14  235.0
  NaN              2.0      2.27708e-11   30.0
 3359.97          33.0    139.818         13.0
   24.6469       500.0      3.77203e-5    33.0
    9.04963       12.0      9.25355        3.0
   88.1082       500.0    NaN             91.0
    8.19584e-33    9.0      9.23872e-21   17.0
    4.04157e5    500.0  85822.2           24.0
    5.02397e-19   27.0    NaN             13.0
    0.00182687   500.0      3.94627e-6    55.0
  482.269        500.0      1.88999e-9   108.0
 2950.85         500.0      1.22637e-5    66.0
    3.9473e-25    58.0      5.51847e-15   18.0
    3.70392e-17   37.0      2.27985e-7    33.0
  NaN              3.0      0.00538632   312.0